<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Get-Best-Substitue" data-toc-modified-id="Get-Best-Substitue-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Get Best Substitue</a></span></li></ul></div>

# Get Best Substitue

In [118]:
clf_RF = RandomForestClassifier(n_estimators=200, criterion='entropy', max_depth=None, min_samples_split=20, ccp_alpha=0.0003, random_state=1234)

In [121]:
clf_RF.fit(X_train, y_train)

RandomForestClassifier(ccp_alpha=0.0003, criterion='entropy',
                       min_samples_split=20, n_estimators=200,
                       random_state=1234)

In [299]:
def getBestSubstitute(model, X, numOfSubs=2 ):
        # get soft prediction
        y_pred_soft=model.predict_proba(X)
        df_proba=pd.concat([X.reset_index(drop=True),pd.DataFrame(y_pred_soft[:,1],columns=['prdictProba'])],axis=1)
        # get mean and std for each  product and Subs
        dfGroup=df_proba.groupby(['ParentProductNumber','ParentProductNumberSubs']).agg({'prdictProba':[('prdictProbaMean','mean'),
                                                                                       ('prdictProbaStd',np.std),
                                                                       ('countRows','count')]}).reset_index()
        dfGroup.columns=dfGroup.columns.droplevel(0)
        dfGroup.columns.values[0]='ParentProductNumber'
        dfGroup.columns.values[1]='ParentProductNumberSubs'
        dfGroup=dfGroup[dfGroup['countRows']>=3]
        # get rank of mean and std
        dfGroup['scoreMean']=dfGroup.groupby('ParentProductNumber')['prdictProbaMean'].apply(lambda x: (x-x.min())/(x.max()-x.min()))
        dfGroup['scoreStd']=dfGroup.groupby('ParentProductNumber')['prdictProbaStd'].apply(lambda x: 1-((x-x.min())/(x.max()-x.min())))
        dfGroup['scoreMeanAndStd']=(dfGroup['scoreMean']*0.8+dfGroup['scoreStd']*0.2).astype(np.float64)
        dfGroup=dfGroup.round({'scoreMeanAndStd':4})
        dfGroup['rankMeanAndStd']=dfGroup.groupby('ParentProductNumber')['scoreMeanAndStd'].rank(ascending=0,method='dense')
        dfGroup['rankMeanAndStd']=np.where(dfGroup['rankMeanAndStd'].isnull(),1,dfGroup['rankMeanAndStd'])
        dfGroup.sort_values(['ParentProductNumber','rankMeanAndStd'],inplace=True)
        # filter best susbs by numofSubs
        dfGroupFinal=dfGroup[(dfGroup['rankMeanAndStd']<=numOfSubs)]
        return dfGroupFinal

In [300]:
df_bestTwoSubs=getBestSubstitute(clf_RF, X)

In [301]:
df_bestTwoSubs.head()

,ParentProductNumber,ParentProductNumberSubs,prdictProbaMean,prdictProbaStd,countRows,scoreMean,scoreStd,scoreMeanAndStd,rankMeanAndStd
12,161631,7290112497918,0.91,0.12,24,1.00,1.00,1.00,1.0
0,161631,161648,0.89,0.17,45,0.94,0.82,0.92,2.0
30,161648,7290112497918,0.91,0.12,31,1.00,1.00,1.00,1.0
16,161648,72909799,0.90,0.15,68,0.60,0.39,0.55,2.0
41,170053,7290104720536,0.78,0.08,11,1.00,0.86,0.97,1.0


In [302]:
df_bestTwoSubs.shape

(935, 9)